In [ ]:
# obtain raw assets
import os
os.system("dvc pull")

In [5]:
# initialize asset runner

import pandas as pd
from asset_runner import AssetRunner

runner = AssetRunner('../data/raw')

In [6]:
# process assets 
runner.process_assets()

--- Processing 2 assets ---
Name         File
-----------  ----------------------------
clubs        ../data/raw/clubs.json
appearances  ../data/raw/appearances.json

-> Processing clubs
       club_id             name domestic_competition
count      241              241                  241
unique     241              241                   14
top        653  vitesse-arnheim                  TR1
freq         1                1                   21
All 1 validations passed!

-> Processing appearances
              goals       assists  ...  appearance_id   season
count  73084.000000  73084.000000  ...   73084.000000  73084.0
mean       0.090676      0.069755  ...   49159.859573   2020.0
std        0.319666      0.272337  ...   28103.821271      0.0
min        0.000000      0.000000  ...     147.000000   2020.0
25%        0.000000      0.000000  ...   25266.750000   2020.0
50%        0.000000      0.000000  ...   48766.500000   2020.0
75%        0.000000      0.000000  ...   73856.250000 

In [7]:
# list available assets
print(runner.prettify_asset_processors())

Name         File
-----------  ----------------------------
clubs        ../data/raw/clubs.json
appearances  ../data/raw/appearances.json


In [8]:
# get a reference to one asset
asset = runner.assets[0]
asset['name']

'clubs'

In [11]:
# get a reference to the pandas dataframes containing raw and prepared data
clubs = asset['processor']

raw = clubs.raw_df
prep = clubs.prep_df

prep.describe()

,club_id,name,domestic_competition
count,241,241,241
unique,241,241,14
top,653,vitesse-arnheim,TR1
freq,1,1,21
